In [1]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import pickle

sys.path.insert(0, '..')
from DataModule.Data_Preparation import CoronnaCERTAINDataset

pd.set_option('display.max_columns', None)

(CVXPY) Jul 05 11:03:01 AM: Encountered unexpected exception importing solver CVXOPT:
ImportError("dlopen(/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so, 0x0002): tried: '/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e'))")
(CVXPY) Jul 05 11:03:01 AM: Encountered unexpected exception importing solver GLPK:
ImportError("dlopen(/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so, 0x0002): tried: '/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e'))")
(CVXPY) Jul 05 11:03:01 AM: Encountered unexpected exception importing solver GLPK_MI:
ImportError("dlopen(/Users/gaskell/miniforge_x86_64/e

In [2]:
dataset = CoronnaCERTAINDataset(
    library_root='/Users/gaskell/Dropbox/Mac/Desktop/Autoimmune_Disease/Code/ML_RA_EHR/Dataset/',
    challenge="regression_delta", #option: regression, regression_delta, classification, binary_classification
    dataset='CORRONA CERTAIN', 
    process_approach='KVB', #option: KVB, SC
    imputation='KNN', #option: SimpleFill, KNN, SoftImpute, BiScaler, NuclearNormMinimization, IterativeImputer, IterativeSVD, None(raw)
    patient_group='bionaive TNF', #option: "all", "bioexp nTNF", "bionaive TNF", "bionaive orencia", "KVB"
    drug_group='all', #option: "all", "actemra", "cimzia", "enbrel", "humira", "orencia", "remicade", "rituxan", "simponi"
    time_points=(0,3), 
    train_test_rate=0.8,
    save_csv=False, 
    random_state=2022)

Int64Index([120, 113, 251, 156, 167, 242, 205, 133,  49, 199,
            ...
            174, 121,  64, 241,  46, 253, 254,  61,  21,  37],
           dtype='int64', length=214)


In [3]:
# read train, test from dataloader
train, train_loc = dataset.get_train()
test, test_loc = dataset.get_test()

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214 entries, 120 to 37
Data columns (total 55 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   3MResponse           214 non-null    float64
 1   grp                  214 non-null    float64
 2   Type I IFN activity  214 non-null    float64
 3   IFNβ activity final  214 non-null    float64
 4   IFNα activity final  214 non-null    float64
 5   IFNβ/α ratio final   214 non-null    float64
 6   DAS28-CRP 3m         214 non-null    float64
 7   age                  214 non-null    float64
 8   gender               214 non-null    float64
 9   final_education      214 non-null    float64
 10  race_grp             214 non-null    float64
 11  ethnicity            214 non-null    float64
 12  weight               214 non-null    float64
 13  height               214 non-null    float64
 14  newsmoker            214 non-null    float64
 15  drinker              214 non-null    fl

In [5]:
train['race_grp'].value_counts()

6.0    193
1.0     10
2.0      5
0.0      3
3.0      3
Name: race_grp, dtype: int64

In [6]:
train.tail()

,3MResponse,grp,Type I IFN activity,IFNβ activity final,IFNα activity final,IFNβ/α ratio final,DAS28-CRP 3m,age,gender,final_education,race_grp,ethnicity,weight,height,newsmoker,drinker,drinksperwk,duration_ra,ndmardused,rfstatus_b,usresultsRF,ccpstatus_b,usresultsCCP3,hxmi,hxunstab_ang,hxchf,hxother_cv,hxtia,hxstroke,hxcopd,hxhtn,hxdiabetes,hxhld,hx_anycancer,seatedbp1_BL,seatedbp2_BL,pres_mtx_BL,pres_arava_BL,pres_azulfidine_BL,pres_plaquenil_BL,pres_imuran_BL,pres_pred_BL,tender_jts_28_BL,swollen_jts_28_BL,md_global_assess_BL,pt_global_assess_BL,di_BL,pt_pain_BL,usresultsCRP_BL,DAS28-CRP_BL,usresultsIgA_BL,usresultsIgG_BL,usresultsIgM_BL,BMI,IFN Missing
253,0.0,3.0,1.091017,0.614766,0.991017,0.620339,1.048870,37.0,0.0,0.0,0.0,1.0,131.0,61.0,1.0,1.0,7.000000,2.0,2.0,1.0,112.3,1.0,251.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,102.0,60.0,1.0,0.0,0.0,1.0,0.0,1.0,9.0,3.0,50.0,75.0,0.00,80.0,1.12,4.445484,302.0,1440.0,119.0,24.749530,0.0
254,1.0,2.0,1.796706,1.693363,1.696706,0.998030,3.647435,55.0,0.0,0.0,6.0,1.0,175.0,60.0,0.0,1.0,0.233333,6.0,2.0,1.0,258.1,1.0,251.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,122.0,80.0,1.0,0.0,0.0,1.0,0.0,1.0,14.0,12.0,65.0,60.0,1.00,20.0,72.19,6.410778,291.0,1470.0,385.0,34.173611,0.0
61,1.0,1.0,0.675392,0.575392,0.575392,1.000000,4.570694,56.0,0.0,1.0,6.0,1.0,146.0,63.0,1.0,0.0,0.000000,1.0,4.0,0.0,11.1,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,146.0,76.0,1.0,1.0,1.0,0.0,0.0,0.0,22.0,6.0,40.0,50.0,0.25,40.0,1.06,5.232664,181.0,983.0,107.0,25.859914,0.0
21,0.0,2.0,1.309226,1.229972,1.209226,1.017156,1.892319,34.0,1.0,2.0,6.0,1.0,220.0,74.0,1.0,0.0,0.000000,6.0,4.0,1.0,22.4,1.0,251.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,138.0,78.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,4.0,30.0,50.0,0.00,50.0,2.91,4.082588,254.0,1370.0,63.0,28.243243,0.0
37,1.0,1.0,2.203191,2.103191,2.103191,1.000000,4.306183,36.0,0.0,1.0,6.0,1.0,189.0,68.0,0.0,1.0,0.233333,1.0,2.0,0.0,10.9,1.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,160.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,6.0,40.0,52.0,0.00,52.0,29.37,5.084322,207.0,1120.0,113.0,28.734213,0.0
